In [113]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
import tempfile
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.enums import TA_LEFT, TA_CENTER
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Image, Spacer
import winsound
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings("ignore")

In [114]:
report_path = "C:\\Users\\Matteo\\Desktop\\eclairys_report.pdf"
benchmark_path = os.getcwd() + '\\CAC 40 - 2025.csv'
benchmark_df = pd.read_csv(benchmark_path)
sector_dict = {
    'Consumer Discretionary' : '^YH102', 
    'Basic Materials' : '^YH101', 
    'Industrials' : '^YH310',
    'Financials' : '^YH103', 
    'Technology' : '^YH311', 
    'Consumer Staples' : '^YH205', 
    'Utilities' : '^YH207',
    'Health Care' : '^YH206', 
    'Telecommunications' : '^YH308',
    'Energy' : '^YH309', 
    'Real Estate' : '^YH104'
}
benchmark_df["Sector ticker"] = benchmark_df["Sector"].map(sector_dict)
benchmark_dict = {
    "Euronext Paris" : "^FCHI",
    "Euronext Amsterdam" : "^FCHI"
}
benchmark_df["Benchmark ticker"] = benchmark_df["Trading Location"].map(benchmark_dict)

In [115]:
stock_history_df = pd.DataFrame()
for ticker in benchmark_df["Ticker"].unique():
    history = yf.Ticker(ticker).history('max')['Close'].tz_localize(None)
    history.name = ticker
    stock_history_df = pd.concat([stock_history_df, history], axis=1)
stock_history_df.index = pd.to_datetime(stock_history_df.index)
stock_history_df = stock_history_df.resample('B').last().ffill()
stock_history_df.index = stock_history_df.index.tz_localize(None)

sector_history_df = pd.DataFrame()
for ticker in benchmark_df['Sector ticker'].unique():
    history = yf.Ticker(ticker).history('max')['Close'].tz_localize(None)
    history.name = ticker
    sector_history_df = pd.concat([sector_history_df, history], axis=1)
sector_history_df.index = pd.to_datetime(sector_history_df.index)
sector_history = sector_history_df.resample('B').last().ffill()
sector_history_df.index = sector_history_df.index.tz_localize(None)

benchmark_history_df = pd.DataFrame()
for ticker in benchmark_df['Benchmark ticker'].unique():
    history = yf.Ticker(ticker).history('max')['Close'].tz_localize(None)
    history.name = ticker
    benchmark_history_df = pd.concat([benchmark_history_df, history], axis=1)
benchmark_history_df.index = pd.to_datetime(benchmark_history_df.index)
benchmark_history_df = benchmark_history_df.resample('B').last().ffill()
benchmark_history_df.index = benchmark_history_df.index.tz_localize(None)

In [116]:
class get_eclairys:
    def __init__(self, stock_history, sector_history, benchmark_history, short_period=[5, 10, 20], long_period=[50, 100, 200]):
        self.short_period = short_period
        self.long_period = long_period
        self.period_list = short_period + long_period
        self.stock_history = stock_history
        self.stock_ticker = stock_history.name
        self.benchmark_history = benchmark_history
        self.benchmark_ticker = self.benchmark_history.name
        self.benchmark_relative_history = self.stock_history / self.benchmark_history
        self.sector_history = sector_history
        self.sector_ticker = self.sector_history.name
        self.sector_relative_history = self.stock_history / self.sector_history

    def calculate_moving_averages(self, data):
        """Calculate moving averages and moving average differentials."""
        MA_df, MAD_df = pd.DataFrame(), pd.DataFrame()
        for period in self.period_list:
            if len(data) < period:
                MA_df[f'MA {period}'] = np.nan
                MAD_df[f'MAD {period}'] = np.nan
            else:
                MA = data.rolling(period).mean()
                MA_df[f'MA {period}'] = MA
                MAD = MA.diff(5)
                MAD_df[f'MAD {period}'] = MAD
        return MA_df, MAD_df

    def normalize_mad(self, MAD_df):
        """Normalize MAD using a rolling window."""
        MADN_df = pd.DataFrame(index=MAD_df.index)
        rolling_max = MAD_df.rolling(200, min_periods=1).max()
        rolling_min = MAD_df.rolling(200, min_periods=1).min()
        MADN_df = 100 * (MAD_df - rolling_min) / (rolling_max - rolling_min)
        MADN_df = MADN_df.replace([np.inf, -np.inf], 0).fillna(0)
        return MADN_df

    def calculate_grade(self, MADN_df):
        """Calculate the GRADE score as the mean of normalized MAD values."""
        GRADE = MADN_df.mean(axis=1)
        return GRADE

    def generate_gps(self, MAD_df, short_period, long_period):
        """Generate GPS indicators based on booleanized MAD values."""
        BMAD = (MAD_df >= 0).astype(int)
        
        # Créer les noms de colonnes pour les périodes courtes et longues
        short_columns = [f'MAD {p}' for p in short_period]
        long_columns = [f'MAD {p}' for p in long_period]
        
        # Calculer la somme des périodes courtes et longues
        SBMAD = BMAD[short_columns].sum(axis=1)
        LBMAD = BMAD[long_columns].sum(axis=1)

        # Générer les indicateurs GPS en fonction des conditions
        conditions = [
            (SBMAD >= 2) & (LBMAD >= 2),
            (SBMAD < 2) & (LBMAD < 2),
            (SBMAD >= 2) & (LBMAD < 2),
            (SBMAD < 2) & (LBMAD >= 2)
        ]
        choices = ['A', 'B', 'C', 'P']
        GPS = pd.Series(np.select(conditions, choices, default=None), index=MAD_df.index)
        return GPS

    def run_analysis(self):
        """Run the entire analysis and return the output DataFrame."""        
        # Calculate indicators for the stock
        stock_MA_df, stock_MAD_df = self.calculate_moving_averages(self.stock_history)
        stock_MADN_df = self.normalize_mad(stock_MAD_df)
        stock_GRADE = self.calculate_grade(stock_MADN_df)
        stock_GPS = self.generate_gps(stock_MAD_df, self.short_period, self.long_period)

        # Combine stock indicators into a DataFrame
        stock_output = pd.concat([
            pd.DataFrame(self.stock_history),
            stock_MA_df,
            pd.DataFrame(stock_GRADE, columns=['GRADE']),
            pd.DataFrame(stock_GPS, columns=['Absolute GPS'])
        ], axis=1)

        # Calculate indicators for the relative trend on sector
        sector_MA_df, sector_MAD_df = self.calculate_moving_averages(self.sector_relative_history)
        sector_GPS = self.generate_gps(sector_MAD_df, self.short_period, self.long_period)
        stock_output['Sector GPS'] = sector_GPS

        # Calculate indicators for the relative trend on Benchmark
        benchmark_MA_df, benchmark_MAD_df = self.calculate_moving_averages(self.benchmark_relative_history)
        benchmark_GPS = self.generate_gps(benchmark_MAD_df, self.short_period, self.long_period)
        stock_output['benchmark GPS'] = benchmark_GPS

        return stock_output
    

In [117]:
# Function to plot the data
def plot_eclairys(df, componenent):
    stock_ticker = df.columns[0]
    plt.rcParams['xtick.labelsize'] = 8
    plt.rcParams['ytick.labelsize'] = 8
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(10, 8), sharex=True, gridspec_kw={'height_ratios': [3, 1, 0.25, 0.25, 0.25], 'hspace': 0.3})

    # 1st plot: Stock Price and Moving Averages
    ax1.plot(df.index, df[stock_ticker], label=stock_ticker, color='black', linewidth=1)
    ax1.plot(df.index, df['MA 5'], label='MA 5', linestyle='--', color='blue', linewidth=0.85)
    ax1.plot(df.index, df['MA 10'], label='MA 10', linestyle='--', color='orange', linewidth=0.85)
    ax1.plot(df.index, df['MA 20'], label='MA 20', linestyle='--', color='green', linewidth=0.85)
    ax1.plot(df.index, df['MA 50'], label='MA 50', linestyle='--', color='red', linewidth=0.85)
    ax1.plot(df.index, df['MA 100'], label='MA 100', linestyle='--', color='purple', linewidth=0.85)
    ax1.plot(df.index, df['MA 200'], label='MA 200', linestyle='--', color='brown', linewidth=0.85)
    ax1.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)
    ax1.set_title(f'{componenent} Price and Moving Averages', fontsize=10)
    ax1.grid(True, alpha=0.5)

    # 2nd plot: GRADE
    ax2.plot(df.index, df['GRADE'], color='black')
    ax2.axhline(y=75, color='green', linewidth=0.75)
    ax2.axhline(y=25, color='red', linewidth=0.75)
    ax2.set_title('GRADE', fontsize=10)
    ax2_legend_elements = [Line2D([0], [0], color='black', lw=1, label='Grade'),
                           Line2D([0], [0], color='green', lw=0.75, label='Buy'),
                           Line2D([0], [0], color='red', lw=0.75, label='Sell')]

    ax2.legend(handles=ax2_legend_elements, loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)
    ax2.grid(True, alpha=0.5)

    # 3rd plot: Absolute GPS with color bands
    gps_colors = {'A': 'green', 'B': 'red', 'P': 'blue', 'C': 'orange'}
    for grade, color in gps_colors.items():
        mask = df['Absolute GPS'] == grade
        ax3.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax3.get_xaxis_transform())
    ax3.set_title('Absolute GPS', fontsize=10)
    ax3.set_yticks([])

    # 4th plot: Benchmark GPS with color bands
    for grade, color in gps_colors.items():
        mask = df['Sector GPS'] == grade
        ax4.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax4.get_xaxis_transform())
    ax4.set_title('Sector GPS', fontsize=10)
    ax4.set_yticks([])

    # 5th plot: benchmark GPS with color bands
    for grade, color in gps_colors.items():
        mask = df['benchmark GPS'] == grade
        ax5.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax5.get_xaxis_transform())
    ax5.set_title('benchmark GPS', fontsize=10)
    ax5.set_yticks([])

    # Add GPS legend
    ax3_legend_elements = [Patch(facecolor='green', label='A', alpha=0.5),
                       Patch(facecolor='red', label='B', alpha=0.5),
                       Patch(facecolor='blue', label='P', alpha=0.5),
                       Patch(facecolor='orange', label='C', alpha=0.5)]
    ax3.legend(handles=ax3_legend_elements, loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)
    plt.tight_layout()
    #plt.show()
    eclairys_chart = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    plt.savefig(eclairys_chart.name, format='png', bbox_inches='tight')
    plt.close()
    return eclairys_chart


In [119]:
buy_list = []
sell_list = []

for stock_ticker, sector_ticker, benchmark_ticker, componenent  in zip(benchmark_df['Ticker'], benchmark_df["Sector ticker"], benchmark_df["Benchmark ticker"], benchmark_df.Component):
    stock_history = stock_history_df[stock_ticker]
    sector_history = sector_history_df[sector_ticker]
    benchmark_history = benchmark_history_df[benchmark_ticker]
    benchmark_ticker = '^GSPC'
    data = get_eclairys(stock_history, sector_history, benchmark_history).run_analysis()
    data = data.iloc[-200:]
    data = data.resample('D').ffill()
    if data.iloc[-1].GRADE > 80:
        chart = plot_eclairys(data, componenent)
        buy_list.append(chart.name)
        print(f'buy: {stock_ticker}')
    elif data.iloc[-1].GRADE < 20:
        chart = plot_eclairys(data, componenent)
        sell_list.append(chart.name)
        print(f'sell: {stock_ticker}')

buy: MT.AS
buy: DSY.PA
buy: EDEN.PA
buy: EL.PA
buy: RMS.PA
buy: GLE.PA


In [ ]:
pdf_file = report_path
doc = SimpleDocTemplate(
    pdf_file, 
    pagesize=letter,
    leftMargin=20,
    rightMargin=20,
    topMargin=20,
    bottomMargin=20
)
title0_style = ParagraphStyle(
    "CustomTitle",
    fontName="Helvetica-Bold",
    fontSize=16,
    leading=12,
    spaceAfter=12,
    alignment=TA_CENTER
)
title1_style = ParagraphStyle(
    "CustomTitle",
    fontName="Helvetica-Bold",
    fontSize=13,
    leading=12,
    spaceAfter=12,
    alignment=TA_LEFT,
    keepWithNext=True
)
bullet0_style = ParagraphStyle(
    "BulletPoint",
    fontName="Helvetica",
    fontSize=10,
    leading=14,
    leftIndent=20
)
bullet1_style = ParagraphStyle(
    "BulletPoint",
    fontName="Helvetica",
    fontSize=9,
    leading=11,
    leftIndent=20
)
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), "#1f77b4"),
    ('TEXTCOLOR', (0, 0), (-1, 0), "#ffffff"),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 6),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 4),
    ('BACKGROUND', (0, 1), (-1, -1), "#ffffff"),
    ('GRID', (0, 0), (-1, -1), 1, "#000000")
])
introduction = "Eclairys est un projet Python permettant d'analyser la dynamique d'une action par rapport à son secteur et au marché global. L'analyse repose sur des moyennes mobiles et des indicateurs de momentum pour évaluer la tendance et la force relative d'un titre."
synthesis = {
    "Report date" : datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "Benchmark" : "CAC 40",
    "benchmark component" : benchmark_df.shape[0],
    "Buy criteria" : "GRADE > 80",
    "Sell criteria" : "GRADE < 20"
}
notes = {
    "MA" : "Moyenne mobile sur 5, 10, 20, 50, 100, 200 du prix de clôture.",
    "GRADE" : "Score absolu basé sur un critère propriétaire.",
    "GPS"   : "Note de 0 à 100 donné en fonction du positionnement du prix et des MA entre eux",
    "Phase Accélération" : "PRIX > MA[5, 10, 20] & PRIX > MA[50, 100, 200]",
    "Phase Baisse" : "PRIX < MA[5, 10, 20] & PRIX < MA[50, 100, 200]",
    "Phase Consolidation" : "PRIX > MA[5, 10, 20] & PRIX < MA[50, 100, 200]",
    "Phase Pause" : "PRIX < MA[5, 10, 20] & PRIX > MA[50, 100, 200]",
}
message_empty_list = "Aucun actif ne respecte les paramètres définis pour cette liste"
elements = [
    Paragraph(f"Eclairys Report", style=title0_style),
    Spacer(1, 10),
    Paragraph(introduction),
    Spacer(1, 10),
    Paragraph("I. Synthèse", style=title1_style), 
    *[Paragraph(f"• {key} : {value}", bullet0_style) for key, value in synthesis.items()],
    Spacer(1, 6),
    *[Paragraph(f"• {key} : {value}", bullet0_style) for key, value in notes.items()],
    Spacer(1, 10),
    Paragraph("II. Signaux d'achat", style=title1_style),
    *([item for chart in buy_list for item in (Image(chart, width=450, height=300), Spacer(1, 10))] if len(buy_list) > 1 else [Paragraph(message_empty_list)]),
    Spacer(1, 10),
    Paragraph("III. Signaux de vente", style=title1_style),
    *([item for chart in sell_list for item in (Image(chart, width=450, height=300), Spacer(1, 10))] if len(sell_list) > 1 else [Paragraph(message_empty_list)]),
    Spacer(1, 10),
    Paragraph("IV. Composition du Benchmark", style=title1_style), 
    Table([benchmark_df.columns.tolist()] + benchmark_df.values.tolist(), style=table_style),
]
total_height = 0
for element in elements:
    w, h = element.wrap(doc.width, doc.height)
    total_height += h
doc = SimpleDocTemplate(
    pdf_file, 
    pagesize=(letter[0], total_height*1.05), 
    leftMargin=20, 
    rightMargin=20, 
    topMargin=20, 
    bottomMargin=20
)
doc.build(elements)
print(f"PDF créé : {report_path}")


PDF créé : C:\Users\Matteo\Desktop\eclairys_report.pdf
